In [18]:
# made by Gibeom LEE, HI LAB
# ImageJ와 같은 영상 분석 툴입니다.
# 민감도를 조절한 후, 각 프레임을 사진으로 저장할 수 있습니다.

# space bar: 일시정지/재시작
# s: 모든 프레임 사진 저장
# q: 종료


# 트랙바 설명
#R_MIN : 픽셀의 빨강 채널 값이 이 값 이상일 때만 선택
#G_MIN : 픽셀의 초록 채널 값이 이 값 이상일 때만 선택
#B_MIN : 픽셀의 파랑 채널 값이 이 값 이상일 때만 선택
#BLUR_K : 영상에 중간값 블러(median blur)를 적용해 노이즈를 줄이는 강도 (커널 크기)
#MORPH_K : 마스크에 열림/닫힘(morphology) 연산을 적용해 작은 잡영역 제거 및 구멍 메우기 강도 (커널 크기)
#LargestOnly : 여러 선택 영역 중 가장 큰 영역만 남길지 여부 (0=모두 유지, 1=가장 큰 것만 유지)
#InvertMask : 마스크를 반전해 선택 영역을 뒤집을지 여부 (0=그대로, 1=반전)
#SideCutPercent : 화면 좌우 몇 %를 잘라내고 중앙 영역만 유지할지 조절 (0=전체, 40=양쪽 40% 잘라냄)

In [19]:
import cv2, time, os, json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import glob  # 폴더 내 파일 리스트 가져오는 라이브러리

# --- 사용자 설정 ---
# 1) 비디오 파일 경로
video_path = Path(r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\251023_LED\FlatDEG_LED.mp4")

# 2) 영상 가로의 실제 길이(mm) → 픽셀-면적 환산용
side_length_mm = 66.0

# --- 저장 프레임 간격 설정 ---
SAVE_EVERY = 1 # 이 값의 배수 프레임만 이미지 저장 (예: 10이면 10,20,30,...)
SAVE_OFFSET = 0       # 시작 오프셋 (예: 5면 5,15,25,...)
CSV_ONLY_SAVED = False  # True면 CSV에도 저장한 프레임만 기록, False면 모든 프레임 기록
EXCLUDE_FRAME0 = True   # ← (NEW) 0번 프레임 저장 제외
DEBUG_PRINT_SAVED = True  # ← (NEW) 저장되는 프레임 인덱스 매번 출력

In [20]:
# ==================================
# 폴더/파일 경로 정의
# ==================================
base_folder = video_path.parent / f"{video_path.stem}_video_analysis"
Original_folder = base_folder / "Original"
Binary_folder = base_folder / "Binary"
csv_path = base_folder / f"{video_path.stem}_area.csv"
params_path = base_folder / "last_params.json"   # 슬라이더 저장 파일

base_folder.mkdir(parents=True, exist_ok=True)

# ==================================
# 핵심 함수 정의 (변화 없음)
# ==================================

def odd_from_slider(x: int) -> int:
    if x <= 1: return 0
    v = x if x % 2 == 1 else x - 1
    return max(3, v)

def win_longpath(path: Path) -> str:
    s = str(path)
    if os.name == "nt":
        if len(s) >= 2 and s[1] == ":" and not s.startswith("\\\\?\\"):
            return "\\\\?\\" + s
    return s

def safe_write_image(path: Path, img) -> bool:
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"[ERR] make dir failed: {e}")
        return False
    ok = cv2.imwrite(win_longpath(path), img)
    if ok:
        return True
    try:
        ext = (path.suffix or ".png").lower()
        if ext not in [".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".tif", ".webp"]:
            ext = ".png"
        ret, buf = cv2.imencode(ext, img)
        if ret:
            buf.tofile(win_longpath(path))
            return True
        else:
            print(f"[ERR] imencode failed for: {path}")
    except Exception as e:
        print(f"[ERR] fallback write failed for {path}: {e}")
    return False

def preprocess_and_mask(frame: np.ndarray, r_min: int, g_min: int, b_min: int, blur_k: int, morph_k: int, largest_only: bool) -> np.ndarray:
    """
    원본 프레임에 전처리(블러링, 모폴로지) 및 색상 필터링을 적용하여 마스크를 생성합니다.
    """
    mask = cv2.inRange(frame, (b_min, g_min, r_min), (255, 255, 255))
    if blur_k > 1:
        mask = cv2.medianBlur(mask, blur_k)
    if morph_k > 1:
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (morph_k, morph_k))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

def postprocess_mask(mask: np.ndarray, invert: bool, side_cut_percent: int, largest_only: bool) -> np.ndarray:
    if invert:
        mask = cv2.bitwise_not(mask)
    h, w = mask.shape[:2]
    cut_percent = np.clip(side_cut_percent, 0, 50) / 100.0
    left_cut = int(w * cut_percent)
    right_cut = int(w * (1.0 - cut_percent))
    roi = np.zeros_like(mask)
    roi[:, left_cut:right_cut] = 255
    mask = cv2.bitwise_and(mask, roi)
    if largest_only:
        fg = (mask > 0).astype(np.uint8)
        num, labels, stats, _ = cv2.connectedComponentsWithStats(fg, connectivity=8)
        if num > 1:
            areas = stats[1:, cv2.CC_STAT_AREA]
            largest = 1 + np.argmax(areas)
            fg = (labels == largest).astype(np.uint8) * 255
            mask = fg
        else:
            mask[:] = 0
    return mask

def overlay_mask(Original_bgr, mask, alpha=0.45):
    color = np.zeros_like(Original_bgr); color[:] = (0, 255, 0)
    mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    return np.where(mask3 > 0,
                    ((1 - alpha) * Original_bgr + alpha * color).astype(np.uint8),
                    Original_bgr)

def put_multiline(img, lines, org=(10, 25), scale=1.0, color=(0, 255, 255),
                  thickness=1, line_gap=30):
    x, y = org
    for line in lines:
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, color, thickness, cv2.LINE_AA)
        y += line_gap
    return img

# ---- 파라미터 저장/복원 (변화 없음) ----
DEFAULT_PARAMS = {
    "R_MIN": 160, "G_MIN": 160, "B_MIN": 140,
    "BLUR_K_RAW": 0, "MORPH_K_RAW": 0,
    "LargestOnly": 1,
    "InvertMask": 1,
    "SideCutPercent": 25,
    "PlaybackSpeedRaw": 10
}
def load_params():
    try:
        with open(win_longpath(params_path), "r", encoding="utf-8") as f:
            d = json.load(f)
        return {**DEFAULT_PARAMS, **d}
    except Exception:
        return DEFAULT_PARAMS.copy()

def save_params(d):
    try:
        params_path.parent.mkdir(parents=True, exist_ok=True)
        tmp = params_path.with_suffix(".json.tmp")
        with open(win_longpath(tmp), "w", encoding="utf-8") as f:
            json.dump(d, f, ensure_ascii=False, indent=2)
        os.replace(win_longpath(tmp), win_longpath(params_path))
    except Exception as e:
        print(f"[WARN] 파라미터 저장 실패: {e}")

# ==================================
# 비디오/스케일 정보 로드 (캡처 객체 1회 생성)
# ==================================
cap_main = cv2.VideoCapture(str(video_path))
if not cap_main.isOpened():
    raise FileNotFoundError(f"영상 열기 실패: {video_path}")
ret, first = cap_main.read() # 첫 프레임 읽어 해상도 확인
if not ret:
    raise RuntimeError("첫 프레임 읽기 실패")

H, W = first.shape[:2]
mm_per_px = side_length_mm / W
area_mm2_per_px = mm_per_px ** 2
total_frames = int(cap_main.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap_main.get(cv2.CAP_PROP_FPS) or 30.0

print(f"[INFO] 해상도 {W}x{H}, 총 프레임 {total_frames}, FPS {fps:.2f}, 1px = {mm_per_px:.5f} mm, 1px² = {area_mm2_per_px:.6f} mm²")

# ==================================
# 트랙바 생성
# ==================================
cv2.namedWindow("Preview", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Preview", min(1500, W * 3), min(900, H * 2))

# --- 전역 변수 설정 ---
paused = False
idx = 0 
trackbar_is_moving = False 
current_frame = first.copy() # 현재 프레임 데이터를 저장할 변수
program_setting_trackbar = False # ⭐ 새로 추가된 플래그: 코드가 트랙바를 설정하는 중인지 확인

def _nothing(_): pass

def on_frame_trackbar(pos):
    """FramePosition 트랙바 콜백: 프레임 이동 시 자동 일시정지"""
    global paused, trackbar_is_moving, program_setting_trackbar
    
    # ⭐ [핵심 수정]: 코드가 설정하는 중이면 무시하고 리턴
    if program_setting_trackbar:
        return
    
    # 트랙바 수동 조작 시 플래그 설정
    trackbar_is_moving = True
        
    # 트랙바를 '수동으로' 조작하면 자동으로 일시정지
    if not paused:
         paused = True
         print(f"[INFO] FramePosition 트랙바 수동 조작 감지. 자동 일시정지(Paused).")
             
cv2.createTrackbar("R_MIN", "Preview", 0, 255, _nothing)
cv2.createTrackbar("G_MIN", "Preview", 0, 255, _nothing)
cv2.createTrackbar("B_MIN", "Preview", 0, 255, _nothing)
cv2.createTrackbar("BLUR_K", "Preview", 0, 31, _nothing)
cv2.createTrackbar("MORPH_K", "Preview", 0, 31, _nothing)
cv2.createTrackbar("LargestOnly", "Preview", 0, 1, _nothing)
cv2.createTrackbar("InvertMask", "Preview", 0, 1, _nothing)
cv2.createTrackbar("SideCutPercent", "Preview", 0, 50, _nothing)
cv2.createTrackbar("PlaybackSpeed", "Preview", 0, 100, _nothing)

max_frame = max(0, total_frames - 1)
cv2.createTrackbar("FramePosition", "Preview", 0, max_frame, on_frame_trackbar)

# ---- 저장된 값으로 초기화 (이전과 동일) ----
p = load_params()
cv2.setTrackbarPos("R_MIN", "Preview", int(p["R_MIN"]))
cv2.setTrackbarPos("G_MIN", "Preview", int(p["G_MIN"]))
cv2.setTrackbarPos("B_MIN", "Preview", int(p["B_MIN"]))
cv2.setTrackbarPos("BLUR_K", "Preview", int(p["BLUR_K_RAW"]))
cv2.setTrackbarPos("MORPH_K", "Preview", int(p["MORPH_K_RAW"]))
cv2.setTrackbarPos("LargestOnly", "Preview", int(p["LargestOnly"]))
cv2.setTrackbarPos("InvertMask", "Preview", int(p["InvertMask"]))
cv2.setTrackbarPos("SideCutPercent", "Preview", int(p["SideCutPercent"]))
cv2.setTrackbarPos("PlaybackSpeed", "Preview", int(p["PlaybackSpeedRaw"]))

# ==================================
# 저장 루틴 (캡처 객체만 수정)
# ==================================
def save_all(R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, LARGEST, INVERT, SIDECUT):
    print("[INFO] 저장 시작...")
    base_folder.mkdir(parents=True, exist_ok=True)
    Original_folder.mkdir(parents=True, exist_ok=True)
    Binary_folder.mkdir(parents=True, exist_ok=True)

    # 저장용 임시 캡처 객체 사용
    cap_save = cv2.VideoCapture(str(video_path))
    if not cap_save.isOpened():
        print("[ERR] 저장용 VideoCapture 열기 실패")
        return

    frame_list, area_px_list, area_mm2_list = [], [], []
    j = 0
    write_fail = 0
    saved_count = 0

    every_n = max(1, int(SAVE_EVERY))
    offset_n = int(SAVE_OFFSET) % every_n

    while True:
        ret2, fr2 = cap_save.read()
        if not ret2:
            break
        
        base_m = preprocess_and_mask(fr2, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, False)
        m2 = postprocess_mask(base_m, invert=INVERT, side_cut_percent=SIDECUT, largest_only=LARGEST)

        px = int(np.count_nonzero(m2))
        
        is_target = ((j - offset_n) % every_n == 0)
        if EXCLUDE_FRAME0 and j == 0:
            is_target = False

        if not CSV_ONLY_SAVED or is_target:
            frame_list.append(j)
            area_px_list.append(px)
            area_mm2_list.append(px * area_mm2_per_px)

        if is_target:
            f_name = f"Original_{j:05d}.png"
            m_name = f"Binary_{j:05d}.png"
            ok1 = safe_write_image(Original_folder / f_name, fr2)
            ok2 = safe_write_image(Binary_folder / m_name, m2)
            if not ok1 or not ok2:
                write_fail += 1
                if not ok1: print(f"[WARN] 원본 저장 실패: {Original_folder / f_name}")
                if not ok2: print(f"[WARN] 마스크 저장 실패: {Binary_folder / m_name}")
            else:
                saved_count += 1
                if DEBUG_PRINT_SAVED:
                    print(f"[SAVE] frame {j} saved (every={every_n}, offset={offset_n})")

        if j % 100 == 0:
            print(f"  processed frame {j}")
        j += 1

    cap_save.release() # 저장용 캡처 객체 닫기

    try:
        df = pd.DataFrame({
            "frame": frame_list,
            "area_px": area_px_list,
            "area_mm2": area_mm2_list
        })
        csv_tmp = base_folder / f"{video_path.stem}_area_tmp.csv"
        df.to_csv(win_longpath(csv_tmp), index=False, encoding="utf-8-sig")
        os.replace(win_longpath(csv_tmp), win_longpath(csv_path))
    except Exception as e:
        print(f"[ERR] CSV 저장 실패: {e}")

    print(f"[INFO] 저장 완료:\n"
          f" - Original: {Original_folder} (saved {saved_count} files)\n"
          f" - Binary  : {Binary_folder} (saved {saved_count} files)\n"
          f" - CSV rows: {len(frame_list)} "
          f"({'only saved frames' if CSV_ONLY_SAVED else 'all frames'})\n"
          f" - Interval: every {every_n} frames (offset {offset_n})\n"
          f" - write_fail: {write_fail}")

# ==================================
# 프리뷰 루프 (최적화 및 버그 수정 로직)
# ==================================

def read_slider_raw():
    return {
        "R_MIN": cv2.getTrackbarPos("R_MIN", "Preview"),
        "G_MIN": cv2.getTrackbarPos("G_MIN", "Preview"),
        "B_MIN": cv2.getTrackbarPos("B_MIN", "Preview"),
        "BLUR_K_RAW": cv2.getTrackbarPos("BLUR_K", "Preview"),
        "MORPH_K_RAW": cv2.getTrackbarPos("MORPH_K", "Preview"),
        "LargestOnly": cv2.getTrackbarPos("LargestOnly", "Preview"),
        "InvertMask": cv2.getTrackbarPos("InvertMask", "Preview"),
        "SideCutPercent": cv2.getTrackbarPos("SideCutPercent", "Preview"),
        "PlaybackSpeedRaw": cv2.getTrackbarPos("PlaybackSpeed", "Preview"),
        "FramePosition": cv2.getTrackbarPos("FramePosition", "Preview")
    }

try:
    while True:
        # 1. 현재 트랙바 위치를 가져옴
        target_idx = cv2.getTrackbarPos("FramePosition", "Preview")
        frame_was_loaded = False
        
        # 2. 프레임 로딩 로직
        if not paused and idx < total_frames - 1:
            # === [재생 중: 순차적 읽기] ===
            ret, frame = cap_main.read()
            if ret:
                current_frame = frame.copy()
                # cap.read() 후 실제 포인터 위치를 idx에 반영
                idx = int(cap_main.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                frame_was_loaded = True
            else:
                # 영상 끝에 도달하면 일시정지 및 트랙바 업데이트
                idx = total_frames - 1 
                program_setting_trackbar = True # ⭐ 트랙바 업데이트 전 플래그 설정
                cv2.setTrackbarPos("FramePosition", "Preview", idx)
                program_setting_trackbar = False # ⭐ 플래그 해제
                paused = True 
                print("[INFO] 영상 끝에 도달했습니다. 일시정지합니다.")
                continue 
                
        elif target_idx != idx:
            # === [트랙바 조작 감지: 점프 로드] ===
            
            # 1. cap_main의 포인터를 트랙바 위치로 명시적으로 재설정
            cap_main.set(cv2.CAP_PROP_POS_FRAMES, target_idx)
            
            # 2. 새로운 프레임을 로드
            ret, frame = cap_main.read()
            if ret:
                current_frame = frame.copy()
                # 3. idx를 트랙바 위치로 강제 동기화 (재생 시작점 확보)
                idx = target_idx 
                frame_was_loaded = True
            else:
                print(f"[WARN] 프레임 {target_idx} 로드 실패.")
                # 실패 시 cap 포인터를 다시 읽어서 idx 보정
                idx = int(cap_main.get(cv2.CAP_PROP_POS_FRAMES)) 
            
            # 트랙바 조작이 완료되었으므로 플래그 해제 (이 플래그는 on_frame_trackbar에서 사용됨)
            trackbar_is_moving = False
            
        elif paused and target_idx == idx:
            # 일시정지 중 & 트랙바 위치와 일치: 프레임 재처리만 수행 (로딩 건너뜀)
            pass
        
        # 3. 슬라이더 값 읽기 및 처리 (이전과 동일)
        raw = read_slider_raw()
        R_MIN = raw["R_MIN"]
        G_MIN = raw["G_MIN"]
        B_MIN = raw["B_MIN"]
        BLUR_K = odd_from_slider(raw["BLUR_K_RAW"])
        MORPH_K = odd_from_slider(raw["MORPH_K_RAW"])
        use_largest = bool(raw["LargestOnly"])
        
        # 재생 속도 계산
        slider_val = raw["PlaybackSpeedRaw"]
        if slider_val == 0:
            playback_speed = 0.1
        else:
            playback_speed = slider_val / 10.0
        
        wait_ms = int((1000 / fps) / playback_speed)
        wait_ms = max(1, wait_ms)
        
        # 4. 마스크 처리 및 오버레이
        base_mask = preprocess_and_mask(current_frame, R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, False)
        mask = postprocess_mask(
            base_mask,
            invert=bool(raw["InvertMask"]),
            side_cut_percent=int(raw["SideCutPercent"]),
            largest_only=use_largest
        )
        
        # 패널 이미지 생성 및 정보 표시
        overlay = overlay_mask(current_frame, mask, alpha=0.45)
        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        panel = np.hstack([current_frame, overlay, mask_bgr])

        lines = [
            f"Frame {idx}/{total_frames-1}",
            "Controls: SPACE=Pause/Play, S=Save, R=Restart, Q=Quit",
            f"Mode: {'PAUSED' if paused else 'PLAYING'}", 
            f"Params: R_MIN={R_MIN} G_MIN={G_MIN} B_MIN={B_MIN} BLUR={BLUR_K} MORPH={MORPH_K} LargestOnly={int(use_largest)}",
            f"Playback Speed: {playback_speed:.1f}x (Wait: {wait_ms}ms)"
        ]

        panel = put_multiline(panel, lines, org=(10, 30))
        cv2.imshow("Preview", panel)
        
        # 5. 키 입력 처리
        key = cv2.waitKey(30 if paused else wait_ms) & 0xFF 
        
        if key == ord('q'):
            save_params(raw)
            break
        elif key == 32:  # SPACE
            # [버그 수정]: 재생 시작/재개 시 cap 포인터를 현재 idx 위치로 재설정
            if paused:
                # 다음 프레임부터 재생되도록, 현재 idx 위치로 설정
                cap_main.set(cv2.CAP_PROP_POS_FRAMES, idx) 
                print(f"[INFO] 재생 시작! cap 포인터 위치를 {idx}로 재설정.")
            
            paused = not paused
            
        elif key == ord('s'):
            print("[INFO] 's' 감지 → 저장을 시작합니다.")
            save_all(R_MIN, G_MIN, B_MIN, BLUR_K, MORPH_K, use_largest,
                     bool(raw["InvertMask"]), int(raw["SideCutPercent"]))
            save_params(raw)
            
        elif key == ord('r'):  # R 키 → 영상 재시작
            print("[INFO] 'r' 감지 → 영상 처음부터 재시작")
            idx = 0
            cap_main.set(cv2.CAP_PROP_POS_FRAMES, idx) 
            program_setting_trackbar = True # ⭐ 트랙바 업데이트 전 플래그 설정
            cv2.setTrackbarPos("FramePosition", "Preview", idx) 
            program_setting_trackbar = False # ⭐ 플래그 해제
            paused = False

        # 6. 트랙바 위치 업데이트 (재생 중일 때만)
        if not paused and frame_was_loaded: 
            if idx < total_frames - 1:
                program_setting_trackbar = True # ⭐ 트랙바 업데이트 전 플래그 설정
                cv2.setTrackbarPos("FramePosition", "Preview", idx)
                program_setting_trackbar = False # ⭐ 플래그 해제
            
finally:
    cap_main.release()
    cv2.destroyAllWindows()
    print("[INFO] 종료")

[INFO] 해상도 798x1332, 총 프레임 359, FPS 24.00, 1px = 0.08271 mm, 1px² = 0.006840 mm²
[INFO] 's' 감지 → 저장을 시작합니다.
[INFO] 저장 시작...
  processed frame 0
[SAVE] frame 1 saved (every=1, offset=0)
[SAVE] frame 2 saved (every=1, offset=0)
[SAVE] frame 3 saved (every=1, offset=0)
[SAVE] frame 4 saved (every=1, offset=0)
[SAVE] frame 5 saved (every=1, offset=0)
[SAVE] frame 6 saved (every=1, offset=0)
[SAVE] frame 7 saved (every=1, offset=0)
[SAVE] frame 8 saved (every=1, offset=0)
[SAVE] frame 9 saved (every=1, offset=0)
[SAVE] frame 10 saved (every=1, offset=0)
[SAVE] frame 11 saved (every=1, offset=0)
[SAVE] frame 12 saved (every=1, offset=0)
[SAVE] frame 13 saved (every=1, offset=0)
[SAVE] frame 14 saved (every=1, offset=0)
[SAVE] frame 15 saved (every=1, offset=0)
[SAVE] frame 16 saved (every=1, offset=0)
[SAVE] frame 17 saved (every=1, offset=0)
[SAVE] frame 18 saved (every=1, offset=0)
[SAVE] frame 19 saved (every=1, offset=0)
[SAVE] frame 20 saved (every=1, offset=0)
[SAVE] frame 21 saved (ev